# Imports

In [1]:
from PIL import Image
import numpy as np
import copy
import cv2 as cv
import os
from scipy.sparse import csr_matrix, hstack
import matplotlib.pyplot as plt
import seaborn as seab
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from sklearn import metrics

In [2]:
all_data = pd.read_csv("../Dataset/PS_20174392719_1491204439457_log.csv")

In [3]:
print len(all_data)
# print type(all_data)

6362620


In [4]:
print all_data.keys()

Index([u'step', u'type', u'amount', u'nameOrig', u'oldbalanceOrg',
       u'newbalanceOrig', u'nameDest', u'oldbalanceDest', u'newbalanceDest',
       u'isFraud', u'isFlaggedFraud'],
      dtype='object')


In [ ]:
for i in range(len(all_data.isFraud)):
    if all_data.isFlaggedFraud[i] == 1 and all_data.isFraud != 1:
        print i